In [1]:
import ray
import time


# A regular Python function.
def normal_task() -> int:
    return 1


# By adding the `@ray.remote` decorator, a regular Python function becomes a Ray remote function.
@ray.remote
def my_function() -> int:
    return 1


# To invoke this remote function, use the `remote` method.
# This will immediately return an object ref (a future) and then create
# a task that will be executed on a worker process.
obj_ref = my_function.remote()

# The result can be retrieved with ``ray.get``.
assert ray.get(obj_ref) == 1


@ray.remote
def slow_function():
    time.sleep(10)
    return 1


# Ray tasks are executed in parallel.
# All computation is performed in the background, driven by Ray's internal event loop.
for _ in range(4):
    # This doesn't block.
    slow_function.remote()


2024-12-13 18:39:28,669	INFO worker.py:1812 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


In [2]:
########## Specifying required resources ##########
@ray.remote(num_cpus=4, num_gpus=2)
def my_function():
    return 1


# Override the default resource requirements.
print(ray.get(my_function.options(num_cpus=2, num_gpus=0).remote()))

1


In [3]:
########## passing object refs to Ray tasks ##########
@ray.remote
def function_with_an_argument(value):
    return value + 1


obj_ref1 = my_function.options(num_cpus=2, num_gpus=0).remote()
assert ray.get(obj_ref1) == 1

# You can pass an object ref as an argument to another Ray task.
obj_ref2 = function_with_an_argument.remote(obj_ref1)
assert ray.get(obj_ref2) == 2


@ray.remote(num_cpus=3)
def slow_function(x: int) -> int:
    print("current is slow_function")
    return x

In [4]:
########## Waiting for Partial Results ##########
time.sleep(10)
object_refs = [slow_function.remote(_) for _ in range(5)]
ready_refs, remaining_refs = ray.wait(object_refs, num_returns=2, timeout=None)  # 阻塞object_refs，直到完成至少2个任务

print(ray.get(object_refs))
print(f"ready refs: {ray.get(ready_refs)}\nremaining refs: {ray.get(remaining_refs)}")

print(f"Waiting for Partial Results Finished")

[0, 1, 2, 3, 4]
ready refs: [0, 1]
remaining refs: [2, 3, 4]
Waiting for Partial Results Finished


In [5]:
########## Multiple returns ##########
# By default, a Ray task only returns a single Object Ref.
@ray.remote
def return_single():
    return 0, 1, 2


object_ref = return_single.remote()
assert ray.get(object_ref) == (0, 1, 2)


# However, you can configure Ray tasks to return multiple Object Refs.
@ray.remote(num_returns=3)  # 可以在修饰中定义return的个数
def return_multiple():
    return 0, 1, 2


object_ref0, object_ref1, object_ref2 = return_multiple.remote()
assert ray.get(object_ref0) == 0
assert ray.get(object_ref1) == 1
assert ray.get(object_ref2) == 2


@ray.remote(num_returns=3)
def return_multiple_as_generator():
    for i in range(3):
        yield i


# NOTE: Similar to normal functions, these objects will not be available
# until the full task is complete and all returns have been generated.
a, b, c = return_multiple_as_generator.remote()

In [7]:
########## Cancelling tasks ##########
@ray.remote
def blocking_operation():
    time.sleep(10e6)


obj_ref = blocking_operation.remote()
ray.cancel(obj_ref)

# try:
#     ray.get(obj_ref)
# except ray.exceptions.TaskCancelledError:
#     print("Object reference was cancelled.")